In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    !pip install openai~=0.28.1
    # !pip install -e $repo_path

import dspy

fatal: cannot change to 'dspy': No such file or directory
Cloning into 'dspy'...
remote: Enumerating objects: 32902, done.
remote: Counting objects: 100% (699/699), done.
remote: Compressing objects: 100% (309/309), done.
remote: Total 32902 (delta 446), reused 592 (delta 376), pack-reused 32203
Receiving objects: 100% (32902/32902), 29.01 MiB | 8.45 MiB/s, done.
Resolving deltas: 100% (15720/15720), done.
Updating files: 100% (4443/4443), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.1/273.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 24.8 MB/s eta 0:00:00
   ━

Define a Dataset

In [ ]:
train = [("sarah palin is awesome. | yes she is", "sarah palin", "positive"),
      ("russell brand and katy perry suit each other. they 're both weird.","katy perry", "negative"),
      ("lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when ...", "lindsay lohan", 'neutral'),
      ('and he has a vision too ! mirzan mahatir is looking like steve jobs right now.', 'steve jobs','neutral'),
      ("i love google ! as a whole with all its services","google","positive")
      ]

# 修改train数据集，使其包含输入`aspect`
train = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in train]

# Check each entry in the dev dataset for errors
dev = [("my 3-year-old was amazed yesterday to find that ' real ' 10 pin bowling is nothing like it is on the wii ...", "wii", "neutral"),
    ("listening 2 '' 3 '' by ms britney spears cant wait 4 the video on oct . 19 you guys need 2 love her new single also buy her single collection nov . 24 !", "britney spears", "positive"),
    ("ff markieai likes animals . hates spiders . hates charlie sheen . likes cookies .", "charlie sheen", "negative"),
    ("espn soccer net news ardiles backs maradona for world cup glory", "maradona", "positive"),
    ("-lsb- happy birthday git * git ! -rsb- ily ! ... some reason im very tired ! but today was really fun ! :-rrb- ugh my ipod died !", "ipod", "neutral"),
    ("hilary swank as amelia earhart is actually pretty great casting .", "hilary swank", "positive"),
    ("video : serena williams gets x-rated for new tennis game ad ! .", "serena williams", "negative"),
    ("lmao i know !! i dnt like harry potter he 's so english !! hehehe its raining here too smdh", "harry potter", "negative"),]
# 修改dev数据集，使其包含输入`aspect`
dev = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in dev]

Define the Metric

In [ ]:
def sentiment_classification_metric(example, pred, trace=None):
    return example.sentiment.lower() == pred.sentiment.lower()

Create the DSPy Intro Context Program

In [ ]:
import dspy
import openai
from dspy import OpenAI

openai.api_base = "https://api.gpts.vin/v1"
openai.api_key = "sk-FhKeNgB7ZgqvpZqsF357D67Eaa8c42339b02Cf57C56a7d19"

lm = OpenAI(api_base=openai.api_base,
             api_key=openai.api_key,
             model='gpt-3.5-turbo')

dspy.settings.configure(lm=lm)

# Define the DSPy implementation of sentiment polarity analysis with an aspect
class IntroSentimentPolarityAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence and its aspect."""
    context = dspy.InputField(desc="Introduction of the aspect within the sentence.")
    sentence = dspy.InputField(desc="The sentence to analyze.")
    aspect = dspy.InputField()
    sentiment = dspy.OutputField(desc="positive, neutral, or negative.")

class IntroSentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.ChainOfThought(IntroSentimentPolarityAnalysis)

    def forward(self, sentence, aspect):
        # Use GPT to create context introduction for the aspect
        context = self.get_aspect_context(aspect)

        prediction = self.analyze_sentiment(context=context, sentence=sentence, aspect=aspect)

        return dspy.Prediction(context=context, sentiment=prediction.sentiment)

    def get_aspect_context(self, aspect):
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Please breifly introduce '{aspect}'."}
            ],
            max_tokens=50,
            n=1,
            stop=None
        )

        context = response.choices[0].message['content'].strip()
        return context

    def analyze_sentiment(self, sentence, aspect, context):
        prompt = (
            f"# Aspect: {aspect}"
            f"# The introduction of '{aspect}': {context}\n"
            f"# Sentence: {sentence}\n"
            f"# Please refer the introduction and analyze the sentiment polarity (positive, neutral, negative) of '{aspect}' in the sentence:\n"
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
            n=1,
            stop=None
        )

        context_sentiment = response.choices[0].message['content'].strip()
        return context_sentiment

Compile the Program

In [ ]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
teleprompter = BootstrapFewShot(metric=sentiment_classification_metric, **config)

# Compile the program
optimized_classifier = teleprompter.compile(IntroSentimentAnalyzer(), trainset=train)

100%|██████████| 5/5 [00:36<00:00,  7.27s/it]


Evaluate the Program

In [ ]:
from dspy.evaluate import Evaluate

# Set up the evaluator
evaluate = Evaluate(devset=dev, metric=sentiment_classification_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate the optimized classifier
evaluate(optimized_classifier)

Average Metric: 4 / 8  (50.0): 100%|██████████| 8/8 [00:21<00:00,  2.64s/it]


50.0

Query the history responses of GPT

In [ ]:
lm.inspect_history(n=1)




Analyze the sentiment polarity of a given sentence and its aspect.

---

Sentence: and he has a vision too ! mirzan mahatir is looking like steve jobs right now.
Aspect: steve jobs
Sentiment: neutral

Sentence: sarah palin is awesome. | yes she is
Aspect: sarah palin
Sentiment: positive

Sentence: lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when ...
Aspect: lindsay lohan
Sentiment: neutral

---

Follow the following format.

Context: Introduction of the aspect within the sentence.

Sentence: The sentence to analyze.

Aspect: ${aspect}

Reasoning: Let's think step by step in order to ${produce the sentiment}. We ...

Sentiment: positive, neutral, or negative.

---

Context: Google is a multinational technology company known for its internet-related services and products, including its search engine, online advertising technologies, cloud computing, software, and hardware. It was founded in 1998 by Larry Page and Se

"\n\n\nAnalyze the sentiment polarity of a given sentence and its aspect.\n\n---\n\nSentence: and he has a vision too ! mirzan mahatir is looking like steve jobs right now.\nAspect: steve jobs\nSentiment: neutral\n\nSentence: sarah palin is awesome. | yes she is\nAspect: sarah palin\nSentiment: positive\n\nSentence: lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when ...\nAspect: lindsay lohan\nSentiment: neutral\n\n---\n\nFollow the following format.\n\nContext: Introduction of the aspect within the sentence.\n\nSentence: The sentence to analyze.\n\nAspect: ${aspect}\n\nReasoning: Let's think step by step in order to ${produce the sentiment}. We ...\n\nSentiment: positive, neutral, or negative.\n\n---\n\nContext: Google is a multinational technology company known for its internet-related services and products, including its search engine, online advertising technologies, cloud computing, software, and hardware. It was 